In [160]:
import csv
import re
from transformers import BertTokenizer

In [161]:
with open('test_input.data', 'r', encoding= 'utf-8') as f:
    data = f.readlines()

print(data[:10])
    

FileNotFoundError: [Errno 2] No such file or directory: 'test_input.data'

In [143]:
class test_output():
    def __init__(self, data):
        
        self.data_list = []
        self.word_id = []
        self.word_article_id = [] 
        article_id = 0
        word_id = 0
        data_list_tmp = []
        article_id_tmp = []
        word_id_tmp = []
        
        for row in data:
            
            data_tuple = tuple()
            if row == '\n':
                
                article_id += 1 
                word_id = 0
                self.word_id.append(word_id)
                self.word_article_id.append(article_id_tmp)
                self.data_list.append(data_list_tmp)
                data_list_tmp = []
                article_id_tmp = []
                word_id_tmp = []

            else:
                
                row = row.strip('\n').split(' ')

                if row[0] in ['。', '？','！','，','～','：',',']:
                    
                    self.word_id.append(word_id_tmp)
                    self.word_article_id.append(article_id_tmp)
                    self.data_list.append(data_list_tmp)
                    data_list_tmp = []
                    article_id_tmp = []
                    word_id_tmp = []
                    
                elif row[0] in ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']:
                      
                    data_tuple = (row[0].lower(), row[1], article_id, word_id)
                    data_list_tmp.append(data_tuple)
                    article_id_tmp.append(article_id)
                    word_id_tmp.append(word_id)

                elif row[0] not in ['摁','嗯','啦','喔','欸','啊','齁','嘿','…','...']:
                    
                    data_tuple = (row[0], row[1], article_id, word_id)
                    data_list_tmp.append(data_tuple)
                    article_id_tmp.append(article_id)
                    word_id_tmp.append(word_id)
                    
                word_id += 1
                
        if len(data_list_tmp) != 0:
            self.data_list.append(data_list_tmp)
            self.word_id.append(word_id_tmp)
            self.word_article_id.append(article_id_tmp)
            
    def raw_output(self):
        return self.data_list, self.word_id, self.word_article_id
    
    def get_stc_label(self):
        
        all_stcs = list()
        all_labels = list()
        all_article_ids = list()
        all_word_ids = list()

        for stc_list in self.data_list:

            txt_len = len(stc_list) #(文章數，每個文章對應的總字數) (word, label)
            stc = str() #存字數= max_stc_len的字串
            labels = []
            article_ids = []
            word_ids = []
            

            for idx, (word, label, article_id, word_id) in enumerate(stc_list):

                stc += word
                labels.append(label)
                article_ids.append(article_id)
                word_ids.append(word_id)

            all_stcs.append(stc)
            all_labels.append(labels)
            all_article_ids.append(article_ids)
            all_word_ids.append(word_ids)
        
        assert len(all_stcs) > 0, 'all stcs len = 0' 
        
        all_stcs_clean = []
        all_labels_clean = []
        all_article_ids_clean = []
        all_word_ids_clean = []
        idx = 0
        
        for stc, label, article_id, word_id in zip(all_stcs, all_labels, all_article_ids, all_word_ids):
            stc_clean = re.sub(r'(醫師)|(個管師)|(民眾)|(家屬)|(護理師)', '', stc)
            # print(stc, stc_clean, label)
            if (len(stc_clean)>=2) & (len(set(label)) >= 2):  
            # print(stc_clean, stc)
                all_stcs_clean.append(stc)
                all_labels_clean.append(label)
                all_article_ids_clean.append(article_id)
                all_word_ids_clean.append(word_id)

            elif (len(stc_clean)>=2) & (((idx+1) % 5) == 0):
                all_stcs_clean.append(stc)
                all_labels_clean.append(label)
                all_article_ids_clean.append(article_id)
                all_word_ids_clean.append(word_id)
            idx += 1

            # 這一步就先把label 做 0 padding
            
        max_length = len(max(all_stcs_clean, key=len))
        assert max_length > 0, 'max length less than 1'
        
        pad_labels = []

        for i in range(len(all_labels_clean)):
            temp_label = ['[PAD]']*max_length
            temp_label[:len(all_labels_clean[i])] = all_labels_clean[i]
            pad_labels.append(temp_label)

        print('sentences總數: {}'.format(len(all_stcs_clean)))
        print('labels總數: {}'.format(len(all_labels_clean)))
            
        return all_stcs_clean, pad_labels, all_article_ids_clean, all_word_ids_clean
    
    def tag2id(self, stcs_label):

        all_label = list()
        for stc_label in stcs_label:
            for label in stc_label:
                all_label.append(label)

        labels_set = sorted(set(all_label))
        tag2id_dict = {'[PAD]':0} #固定將PAD id設為0

        labels_set.remove('[PAD]')

        for idx, label in enumerate(labels_set):
            tag2id_dict[label] = idx+1

        return tag2id_dict

    def label_to_ids(self, tag_to_id, raw_labels):

        label2id = []
        for stc_labels in raw_labels:
            stc_label_ids = [tag_to_id[label] for label in stc_labels]
            label2id.append(stc_label_ids)
        return label2id

    def get_stcs_label2ids(self):

        stcs, labels, article_ids, word_ids = self.get_stc_label()
        tag2id = self.tag2id(stcs_label= labels)
        labels_ids= self.label_to_ids(tag_to_id= tag2id, raw_labels= labels)

        return stcs, labels_ids, article_ids, word_ids

In [144]:
stcs, pad_labels, article_ids, word_ids = test_output(data= data).get_stcs_label2ids()

sentences總數: 6069
labels總數: 6069


In [145]:
stcs

['還是虛虛的',
 '阿只是前天好很多',
 '前天就算沒盜',
 '一覺到天明',
 '它有幫你驗了不少',
 '我相信你之前都驗過',
 '不過你工作壓力很大',
 'q熱陰性',
 '然後恙蟲病',
 '可以',
 '恙蟲是在離島比較多',
 '了解了解',
 'ebv還有cmv',
 '這是良性的',
 '阿這也是都沒有',
 'crp',
 '85',
 '102',
 '這個可能是沒睡好',
 '喝酒',
 '你說假如是前年',
 '但是去年可能',
 '就沒有',
 '我前年都在對岸比較多',
 '前年',
 'google有設廠',
 '客戶在那邊',
 '阿也有設廠',
 'google不是',
 'abs',
 '就是有些人確實會有一些',
 '就是這次照起來的話就是',
 '你三月十八住院的時候',
 '腸壁比較厚一點',
 '你看厚厚的',
 '那厚很多',
 '所以是發炎',
 '這次是剛好發炎',
 '我覺得一開始應該是這樣',
 '但是',
 '哨站',
 '這個你局部這個地方',
 '就會阿浪',
 '就會過來這樣子',
 '白血球嗎',
 '理論上應該要',
 '阿不過我必須',
 '放射科醫師他覺得這個是',
 '這種是良性的',
 'reactive',
 '了解',
 '會比這個還大',
 '比較怪異',
 '我想我們計劃就看',
 '這樣就好了',
 '兩個月之後再追蹤看看',
 '沒有問題',
 '阿賈伯斯的你看了嗎',
 '賈伯斯的',
 '他是下下禮拜',
 '他是下下禮拜',
 '你住院三月二十一到現在',
 '那我們再兩個禮拜',
 '我想抗生素還是讓你吃一下好了',
 '還是有吃',
 '綠色有吃',
 '看一下',
 '真的',
 '我都可以',
 '再吃兩個禮拜',
 '對對對',
 '綠色的一定要吃',
 '還是少',
 '而且血液培養也沒有長細菌',
 '那你今天是請假還是',
 '我後天上班',
 '畢竟',
 '畢竟那個',
 '阿你定居美國這樣',
 '我現在住',
 '住紐約那邊',
 '紐約',
 '有一個新總圖',
 '我也是這樣想',
 '賈伯斯下下禮拜有抽血',
 '也可以',
 '普拿疼都還有嗎',
 '都還有',
 '普拿疼我要吃到什麼時候會停',
 '就是不用說那個',
 '了解'

In [107]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [146]:
max_len = max(len(txt) for txt in stcs)

In [147]:
print(len(stcs))

6069


In [152]:
encoding = tokenizer.batch_encode_plus(stcs, 
                                 padding=True,
                                 add_special_tokens=False,
                                 return_attention_mask= True,
                                 return_token_type_ids= False,
                                 is_split_into_words=True,
                                 return_tensors='pt')

In [153]:
encoding['input_ids'].size()

torch.Size([6069, 39])

In [154]:
encoding['attention_mask'].size()

torch.Size([6069, 39])

In [157]:
encoding['attention_mask'][:2]

tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
# loss, preds = model(input_ids, 

-----

In [ ]:
stcs, pad_labels, article_ids, word_ids